В якості домашнього завдання вам пропонується створити нейронну мережу за допомогою механізмів Keras, яка буде __класифікувати__ товари із датасету `fasion_mnist`.

Вам належить запропонувати свою власну архітектуру мережі. Точність найнаївнішої, але адекватної нейромережі становить приблизно __91%__. Точність вашої моделі повинна бути `не нижчою` за цей показник. Щоб досягти таких значень вам знадобиться `поекспериментувати з гіперпараметрами мережі`:

- кількість шарів;

- кількість нейронів;

- функції активації;

- кількість епох;

- розмір батчу;

- вибір оптимізатора;

- різні техніки регуляризації і т.д.

Використайте вивчені `техніки виявлення проблем навчання` нейронної мережі, і потім `поекспериментуйте`.

Рішення оформіть у вигляді окремого ноутбука.

In [214]:
import itertools
from typing import Optional, Union

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.activations import mish, relu, sigmoid, softmax
from tensorflow.keras.datasets import fashion_mnist
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras import losses
from keras import metrics
from keras import optimizers

# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.metrics import accuracy_score
# import tensorflow as tf
# from tensorflow.keras.activations import sigmoid
# from tensorflow.keras.datasets import mnist
# from tensorflow.keras.metrics import Accuracy
# from tensorflow.keras.utils import to_categorical

###### __Dataset__

In [149]:
# learning_rate = 0.001  # скорость обучения нейронной сети
# batch_size = 256  # пересчитывать веса сети мы будем не на всей выборке, а на ее случайном подможестве из batch_size элементов

# n_hidden_1 = 128  # количество нейронов 1-го слоя
# n_hidden_2 = 256  # количество нейронов 2-го слоя

# BUFFER_SIZE = 5000  # This dataset fills a buffer with buffer_size elements, then randomly samples elements from this buffer, replacing the selected elements with new elements.

In [150]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train.dtype, x_train[0].shape

(dtype('uint8'), (28, 28))

In [151]:
NUM_FEATURES = x_train[0].shape[0] * x_train[0].shape[1]

In [152]:
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

# each sample (28x28) to 1x784
x_train, x_test = x_train.reshape([-1, NUM_FEATURES]), x_test.reshape([-1, NUM_FEATURES])
print(min(x_train.min(), x_test.min()))
maximum = max(x_train.max(), x_test.max()) # np.max([x_train.max(), x_test.max()]) 
maximum

0.0


255.0

In [184]:
# Normalize (min = 0)
x_train, x_test = x_train / maximum, x_test / maximum

In [154]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((60000, 784), (10000, 784), (60000,), (10000,))

In [155]:
max(y_train.max(), y_test.max()), min(y_train.min(), y_test.min())

(9, 0)

In [157]:
NUM_CLASSES = len(set(np.concatenate((y_train, y_test), axis=0)))
NUM_CLASSES

10

In [203]:
y_true = tf.one_hot(y_train, depth=NUM_CLASSES).numpy()
y_truev = tf.one_hot(y_test, depth=NUM_CLASSES).numpy()
y_true.shape, y_truev.shape

((60000, 10), (10000, 10))

###### __Model__

- https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
- https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy

In [222]:
# number of hidden layers
n_layers = [el for el in range(1, 4)]  # 1, 2, 3
n_of_neurons = [2**el for el in range(1, 9)] # [[2**el for el in range(1, 9)] for _ in n_layers]  # 8

activations = ['mish', 'relu', 'sigmoid', 'softmax']

optimizators = [optimizers.legacy.SGD, optimizers.legacy.RMSprop, optimizers.legacy.Adam]  # 3
learning_rates = [10**el for el in range(-3, 0)]  # 3
n_epoch = 100  # max limit
batch_sizes = [2**el for el in range(7, 10)]  # 3



In [237]:
n_hiddens_ = []
bundle_activations  = []
for s in n_layers:
    for el in list(itertools.product(n_of_neurons, repeat=s)):
        n_hidden_i = []
        n_hidden_i.append(NUM_FEATURES)
        n_hidden_i.extend(el)
        n_hidden_i.append(NUM_CLASSES)

        n_hiddens_.append(n_hidden_i)

    for el in list(itertools.product(activations[:-1], repeat=s)):
        activat = []
        activat.extend(el)
        activat.append(activations[-1])

        bundle_activations.append(activat)
    

In [238]:
len(n_hiddens_), len(bundle_activations)

(584, 39)

In [246]:
cx = 0
for set_layers in n_hiddens_:  # 584
    for act in activations:  # 39
        if len(set_layers) == (len(act) + 1):
            cx+=1

cx

1024

In [258]:
def create_NN(n_hidden_: list, activation: list, optimizer=optimizers.SGD, learning_rate=0.1) -> Sequential:
    model = Sequential(name=f'''m-{[f'{i}-' for i in n_hidden_[1:-1]]}{[f'{a}-' for a in activation[:-1]]}{optimizer.__name__}-{learning_rate}''')
    n_layers = len(n_hidden_)
    for layer in range(1, n_layers):
        model.add(Dense(n_hidden_[layer], activation=activation[layer-1], input_shape=(n_hidden_[layer-1],)))

    model.compile(
                  optimizer=optimizer(learning_rate=learning_rate), # legacy
                  loss=losses.CategoricalCrossentropy(),  # 'categorical_crossentropy' losses.binary_crossentropy,
                  metrics=['accuracy']  # metrics.BinaryAccuracy()
                  )

    return model

In [259]:
models = []  # 9216  # !!! more then 32 GB RAM!!!!
for set_layers in n_hiddens_:  # 584
    for act in bundle_activations:  # 39
        if len(set_layers) == (len(act) + 1):  # 1024
            for opt in optimizators:  # 3
                for lr in learning_rates:  # 3
                    models.append(create_NN(n_hidden_=set_layers, activation=act, optimizer=opt, learning_rate=lr))

###### __Training__

In [ ]:
history = {}  # 27648 = 38+ days training
for model in models:
    for bs in batch_sizes:  # 3
        history[f'{model.name}({bs})'] = model.fit(
                                                    x_train,
                                                    y_true,
                                                    epochs=100,
                                                    batch_size=bs,
                                                    validation_data=(x_test, y_truev)
                                                    )

###### __Error Analysis__

In [204]:
# print(history[0].history.keys())
""" 
loss - список, в якому зберігається історія змін помилки на навчальних даних залежно від епохи
val_loss - список, де зберігається історія змін помилки на тестових даних залежно від епохи
accuracy - список, в якому зберігається історія змін точності на навчальних даних залежно від епохи
val_accuracy - список, де зберігається історія змін точності на тестових даних залежно від епохи
"""

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])


' \nloss - список, в якому зберігається історія змін помилки на навчальних даних залежно від епохи\nval_loss - список, де зберігається історія змін помилки на тестових даних залежно від епохи\naccuracy - список, в якому зберігається історія змін точності на навчальних даних залежно від епохи\nval_accuracy - список, де зберігається історія змін точності на тестових даних залежно від епохи\n'

In [213]:
def draw_2twin_2d(y00: np.array, y01: np.array, y10: np.array, y11: np.array, suptitle: str) -> None:
    x = range(1, len(y00) + 1)
    
    fig, axs = plt.subplots(1, 2, figsize=(12, 8))

    axs[0].plot(x, y00, 'bo', label='Training loss')
    axs[0].plot(x, y01, 'g', label='Validation loss')
    axs[0].title.set_text('Training and validation loss')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Loss')
    axs[0].legend()
    axs[0].grid()

    axs[1].plot(x, y10, 'bo', label='Training acc')
    axs[1].plot(x, y11, 'g', label='Validation acc')
    axs[1].title.set_text('Training and validation acc')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Acc')
    axs[1].legend()
    axs[1].grid()
    
    fig.suptitle(suptitle)
    # plt.legend()
    # plt.grid()
    axs[1].grid()
    plt.show()


In [ ]:
for model in history:
    history_dict = history[model].history

    loss_values = history_dict['loss']
    val_loss_values = history_dict['val_loss']
    acc_values = history_dict['accuracy']
    val_acc_values = history_dict['val_accuracy']   

    draw_2twin_2d(loss_values, val_loss_values, acc_values, val_acc_values, suptitle=model)
